#### Objective

This notebook shows an example of how to:
- Use Welly to load two wells with Vp, Vs, and RHOB curves
- store the wells in Pandas DataFrames, the nconcatenate in a single Pandas DataFrame 
- filter and upscale with Backus average all curves individually, well by well, using Padas groupby 

#### Import libraries

In [ ]:
import bruges as br
import numpy as np
import pandas as pd
from welly import Well
import matplotlib.pyplot as plt

#### Import wells

In [ ]:
R39 = Well.from_las('../data/R-39.las')

In [ ]:
w129 = Well.from_las('../data/P-129_out.las')

#### Data clean-up and manipulation

- Make dataframes
- Deal with null values
- Select columns of interest
- Convert slowness to velocity
- Add well name column
- Concatenate dataframes

Make dataframes

In [ ]:
w39_df = R39.df()
w39_df = w39_df[['DT4P', 'DT4S', 'RHOB']]
w39_df.columns = ['DT', 'DTS', 'RHOB']
w39_df.describe(include = 'all')

Checking for null values

In [ ]:
for x in w39_df.columns:
  print (x, w39_df[x].isnull().values.any())

print(w39_df.isnull().sum()) # finds how many missing points there are

There are no nulls in R-39

In [ ]:
w129_df = w129.df()
w129_df = w129_df[['DT', 'DTS', 'RHOB']]
w129_df.describe(include = 'all')

In [ ]:
for x in w129_df.columns:
  print (x, w129_df[x].isnull().values.any())

print(w129_df.isnull().sum()) # finds how many missing points there are

It looks like there are several null values in P-129. Below we replace missing values with the mean, then check again

In [ ]:
w129_df['DT'].fillna(w129_df['DT'].mean(),inplace=True)
w129_df['DTS'].fillna(w129_df['DTS'].mean(),inplace=True)
w129_df['RHOB'].fillna(w129_df['RHOB'].mean(),inplace=True)

for x in w129_df.columns:
  print(x, w129_df[x].isnull().values.any())

Convert slowness to velocity

For well P-129: usec/ft >> m/s

In [ ]:
w129_df['Vp'] = 3.048e5 / w129_df['DT'] 
w129_df['Vs'] = 3.048e5 / w129_df['DTS'] 
w129_df.describe(include = 'all')

For well -39: usec/m >> m/s

In [ ]:
w39_df['Vp'] = 1.0e6 / w39_df['DT'] 
w39_df['Vs'] = 1.0e6 / w39_df['DTS'] 
w39_df.describe(include = 'all')

Add well name columns

In [ ]:
w39_df['DEPTH'] = w39_df.index
name = (['R-39']*len(w39_df))
w39_df['well'] = name
w39_df = w39_df.reset_index(drop=True)
w39_df.describe(include = 'all')

In [ ]:
w129_df['DEPTH'] = w129_df.index
name = (['P-129']*len(w129_df))
w129_df['well'] = name
w129_df = w129_df.reset_index(drop=True)
w129_df.describe(include = 'all')

Concatenate dataframes drop slownness columns, sort columns

In [ ]:
wells = pd.DataFrame()

for i,df in enumerate([ w129_df, w39_df]):
    wells=wells.append(df)   

wells.drop(wells.columns[[0, 1]], inplace=True, axis=1) 
wells = wells[['DEPTH', 'Vp', 'Vs', 'RHOB', 'well']]    
wells.describe(include = 'all')

#### Despike Vp, Vs, and RHOB curves

##### Logic:
    group dataframe by well

        for each well

            for each of Vp, Vs, RHOB
    
                smooth log using Bruges' median

In [ ]:
logs = ['Vp', 'Vs', 'RHOB']

In [ ]:
window = 11

In [ ]:
wells_sm = pd.DataFrame()

grouped = wells['well'].unique()

for well in grouped:    
    new_df = pd.DataFrame()   
    for log in logs:
        sm = br.filters.mean(arr=pd.Series(wells[log][wells['well'] == well]), size= window)
        new_df[str(log) + '_sm'] = sm 
    wells_sm = pd.concat([wells_sm, new_df])

Combine DataFrame of smoothed curves with original DataFrame

In [ ]:
wells_despiked = (np.concatenate((wells.values, wells_sm.values), axis=1))
cols = list(wells) + list(wells_sm)
wells_despiked_df = pd.DataFrame(wells_despiked, columns=cols)
wells_despiked_df.describe()

Make sure results make sense

In [ ]:
fig = plt.figure(figsize=(6,10))
ax0 = fig.add_subplot(121)
ax0.plot(wells_despiked_df.loc[wells_despiked_df.well == 'P-129', 'Vp'], 
         wells_despiked_df.loc[wells_despiked_df.well == 'P-129', 'DEPTH'],
                                 'gray', alpha=0.6, linewidth = 3, label='Vp')
ax0.plot(wells_despiked_df.loc[wells_despiked_df.well == 'P-129', 'Vp_sm'], 
         wells_despiked_df.loc[wells_despiked_df.well == 'P-129', 'DEPTH'],
                                 'k', linewidth = 1, label='despiked Vp')
ax0.set_xlim([2500, 7500])
ax0.legend( loc='upper left')
ax0.set_title('Despiked Vp')
ax0.set_ylabel(r'depth [m]', size=12)
ax0.invert_yaxis()
ax0.set_xlabel(r'Vp [m/s]', size=12)

ax1 = fig.add_subplot(122)
ax1.plot(wells_despiked_df.loc[wells_despiked_df.well == 'P-129', 'Vs'], 
         wells_despiked_df.loc[wells_despiked_df.well == 'P-129', 'DEPTH'],
                                 'lightblue', alpha=0.6, linewidth = 3, label='Vs')
ax1.plot(wells_despiked_df.loc[wells_despiked_df.well == 'P-129', 'Vs_sm'], 
         wells_despiked_df.loc[wells_despiked_df.well == 'P-129', 'DEPTH'],
                                 'b', linewidth = 1, label='despiked Vs')
ax1.set_xlim([1500, 4000])
ax1.legend( loc='upper left')
ax1.set_title('Despiked Vp')
ax1.set_ylabel(r'depth [m]', size=12)
ax1.invert_yaxis()
ax1.set_xlabel(r'Vp [m/s]', size=12)

plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,10))
ax0 = fig.add_subplot(121)
ax0.plot(wells_despiked_df.loc[wells_despiked_df.well == 'R-39', 'Vp'], 
         wells_despiked_df.loc[wells_despiked_df.well == 'R-39', 'DEPTH'],
                                 'gray', alpha=0.6, linewidth = 3, label='Vp')
ax0.plot(wells_despiked_df.loc[wells_despiked_df.well == 'R-39', 'Vp_sm'], 
         wells_despiked_df.loc[wells_despiked_df.well == 'R-39', 'DEPTH'],
                                 'k', linewidth = 1, label='despiked Vp')
ax0.set_xlim([2500, 7500])
ax0.legend( loc='upper left')
ax0.set_title('Despiked Vp')
ax0.set_ylabel(r'depth [m]', size=12)
ax0.invert_yaxis()
ax0.set_xlabel(r'Vp [m/s]', size=12)

ax1 = fig.add_subplot(122)
ax1.plot(wells_despiked_df.loc[wells_despiked_df.well == 'R-39', 'Vs'], 
         wells_despiked_df.loc[wells_despiked_df.well == 'R-39', 'DEPTH'],
                                 'lightblue', alpha=0.6, linewidth = 3, label='Vs')
ax1.plot(wells_despiked_df.loc[wells_despiked_df.well == 'R-39', 'Vs_sm'], 
         wells_despiked_df.loc[wells_despiked_df.well == 'R-39', 'DEPTH'],
                                 'b', linewidth = 1, label='despiked Vs')
ax1.set_xlim([1500, 4000])
ax1.legend( loc='upper left')
ax1.set_title('Despiked Vp')
ax1.set_ylabel(r'depth [m]', size=12)
ax1.invert_yaxis()
ax1.set_xlabel(r'Vs [m/s]', size=12)

plt.tight_layout()
plt.show()

#### Upscale with Backus average Vp, Vs, and RHOB

##### To be continued...